# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с помощью Selenium: пример ВКонтакте

Импортируем из библиотеки `selenium` модуль `webdriver` для доступа к браузеру:

In [1]:
from selenium import webdriver as wd

Откроем новое окно браузера (укажите свой путь к файлу с драйвером для Chrome):

In [2]:
br = wd.Chrome("/Users/allat/Downloads/chromedriver")

В открывшемся окне в адресную строку запишем ссылку на главную страницу социальной сети:

In [3]:
br.get("https://vk.com/")

Найдём поле для ввода логина и введём туда свой логин. Чтобы понять, по какому тэгу/иному идентификатору обнаружить это поле, обратимся к исходному коду страницы. Заметим, что у поля для логина есть id – отлично, значит, вызывать его можно однозначным образом, все id уникальны:

In [4]:
login = br.find_element_by_id("index_email")
login.send_keys("allatambov@mail.ru")

Поле для пароля найдём аналогичным образом:

In [5]:
password = br.find_element_by_id("index_pass")

С вводом пароля поступим хитро – импортируем функцию `getpass()` из модуля `getpass()`, которая запрашивает пароль с клавиатуры и скрывает введённые символы:

In [6]:
from getpass import getpass

In [7]:
my_password = getpass()

········


А теперь отправим это значение в соответствующее поле:

In [8]:
password.send_keys(my_password)

Осталось кликнуть на кнопку для входа! Тоже найдём её по id:

In [9]:
go = br.find_element_by_id("index_login_button")
go.click() 

Теперь, когда мы залогинены и действуем со своего аккаунта, мы можем приступить к работе. Перейдём на страницу поиска друзей:

In [10]:
br.get("https://vk.com/friends")

Найдём на этой странице ссылку с текстом *Поиск друзей*, чтобы перейти к странице поиска пользователей по заданным критериям, и кликнем на неё:

In [11]:
search = br.find_element_by_link_text("Поиск друзей")
search.click() 

Чтобы формировать критерии для поиска, нам нужно открыть меню с фильтрами – развернуть меню *Параметры поиска*. Изучив исходный код страницы, видим, что параметры поиска можно найти по id:

In [12]:
pars = br.find_element_by_id("friends_filters_block")
pars.click()

Для начала выберем страну (регион). В исходном коде страницы поле для ввода страны имеет тэг `<input>` и id, равный `country`. Но давайте для разнообразия найдём это поле не по id, а более сложным, двухступенчатым способом, чтобы увидеть, как реализуется поиск элементов, которые вложены в другие. Раздел, который соответствует стране, имеет id, равный `cCountry`:

In [13]:
cnt = br.find_element_by_id("cCountry")

Объект `cnt` – это объект типа `webelement.WebElement`, то есть элемент страницы, с которым умеет работать модуль `webdriver` из `selenium`. По такому объекту тоже можно выполнять поиск с помощью методов вида `find_element_by_`. 

В нашем случае внутри этого элемента нужно найти другой, с тэгом `<input>`, потому что нас интересует поле для ввода значения. Если этим шагом пренебречь и попытаться ввести название страны прямо в `cnt`, мы получим ошибку вида `element not interactable`, потому что сам по себе раздел со страной никакого взаимодействия с пользователем не предполагает, его нельзя редактировать, на него нельзя кликать и прочее.

Поэтому найдём внутри `cnt` поле для ввода значения по тэгу:

In [14]:
cnt_inp = cnt.find_element_by_tag_name("input")
cnt_inp.send_keys("Россия") 

Отлично! Значение выбрано. Но есть проблема – оно «повисло» в воздухе, опция с выбором страны отображается как выбранная в выпадающем меню, но в самом поле выбор не зафиксирован. Чтобы подтвердить выбор, нужно нажать на клавишу *Enter*. К счастью, `selenium` умеет имитировать действия клавиатуры и мышки. Импортируем «хранилище» клавиш `Keys`:

In [15]:
from selenium.webdriver.common.keys import Keys

И теперь «введём» в поле для названия страны имитацию нажатия клавиши *Enter* (*Return*):

In [16]:
cnt_inp.send_keys(Keys.RETURN)

Отлично! Продолжим работу с фильтрами и проделаем то же самое для выбора города, только с маленькой поправкой – так как ниже код уже готовый и запускается одним разом, а не поэтапно, чтобы необходимая информация успела прогрузиться на странице, перед нажатием на *Enter* немного подождём: 

In [17]:
city = br.find_element_by_id("cCity")
city_inp = city.find_element_by_tag_name("input")
city_inp.send_keys("Москва") 

In [18]:
br.implicitly_wait(3)  # задержка
city_inp.send_keys(Keys.RETURN)

А вот с полом всё поинтереснее: найти поле для ввода пола просто, а вот значения нужно выбирать, нажимая на радиокнопки (*radiobuttons*). Сначала найдём поле для выбора пола:

In [19]:
sex = br.find_element_by_id("cSex")

А теперь – все опции внутри (согласно исходному коду, они имеют тэг `<div>`):

In [20]:
values = sex.find_elements_by_tag_name("div")
values

[<selenium.webdriver.remote.webelement.WebElement (session="486d62f41fc38231d3ee4ab05f878175", element="0.10024155086720699-8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="486d62f41fc38231d3ee4ab05f878175", element="0.10024155086720699-9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="486d62f41fc38231d3ee4ab05f878175", element="0.10024155086720699-10")>]

Обратите внимание: здесь метод `find_elements_`, не `find_element_`, потому что результатов ожидается несколько. Методы вида `find_element_` возвращают только первое совпадение на странице, методы вида `find_elements_` – все совпадения на странице (можно провести аналогию с `find` и `find_all` в `BeautifulSoup`).

Выбираем мужской пол – это второй элемент списка – и кликаем на него:

In [21]:
values[1].click()

На этом закончим работу с фильтрами и перейдём к результатам. На этом этапе возможности `selenium` нам пока не понадобятся, нам нужно только запросить исходный код страницы, которая сейчас открыта в окне браузера, управляемом из Python, и продолжить работу с HTML с помощью BeautfulSoup.

In [22]:
from bs4 import BeautifulSoup

# забираем исходный код страницы
# превращаем в объект BeautifulSoup

html = br.page_source
soup = BeautifulSoup(html)

Если мы внимательно изучим исходный код страницы с результатами в браузере, мы заметим, что краткая информация по пользователям (имя и ссылка на профиль) находится в тэгах `<div>` с классом `info`:

In [23]:
divs = soup.find_all("div", {"class" : "info"})

Возьмём первого пользователя и поймём, как извлечь информацию по нему:

In [24]:
d = divs[0]
d

<div class="info">
<div class="labeled name"><a href="/temalebedev" onclick="return nav.go(this, event);">Артемий Лебедев</a><a class="page_verified" href="/verify" onmouseover="pageVerifiedTip(this, {type: 1, oid: 18425009})"></a></div>
<div class="friends_user_info_actions"><a class="friends_field_act" href="/write18425009" onclick="return showWriteMessageBox(event, 18425009)">Написать сообщение</a></div>
</div>

Найдём элемент, соответствующий имени и извлечём оттуда текст и ссылку на профиль:

In [25]:
person = d.find("div", {"class" : "labeled name"})
name = person.text
href = person.find("a")["href"]

Напишем функцию для извлечения имени и ссылки на профиль:

In [26]:
def get_person(d):
    person = d.find("div", {"class" : "labeled name"})
    name = person.text
    href = person.find("a")["href"]
    return name, href

И применим её ко всем пользователям в списке `divs`:

In [27]:
people = []
for d in divs:
    p = get_person(d)
    people.append(p)

In [28]:
people

[('Артемий Лебедев', '/temalebedev'),
 ('Jama Buldozer', '/jama_buldozer'),
 ('Павел Дуров', '/durov'),
 ('Вован Тарасов', '/id579521433'),
 ('Сергей Крылов', '/id11010073'),
 ('Ян Топлес', '/yanlapotkov'),
 ('Сергей Миронов', '/mironov_sergey'),
 ('Александр Кондрашов', '/justdoit150'),
 ('Денис Джиган', '/iamgeegun'),
 ('Стас Васильев', '/stas_satori'),
 ('Амиран Сардаров', '/amirans'),
 ('Константин Малофеев', '/kvmalofeev'),
 ('Митя Фомин', '/id87211536'),
 ('Сергей Домогацкий', '/domogatskiy'),
 ('Стас Костюшкин', '/stas_kostyushkin'),
 ('Сергей Жуков', '/sergeyzhukov.official'),
 ('Роберт Григорян', '/scorty'),
 ('Александр Соколовский', '/id8006'),
 ('Олег Абрамов', '/tapog'),
 ('Андрей Воробьев', '/andreyvorobiev'),
 ('Тимур Батрутдинов', '/batr'),
 ('Ранис Гайсин', '/ranto'),
 ('Алексей Русских', '/russkihay'),
 ('Мария Ивакова', '/maria_ivakova'),
 ('Борис Александрович', '/borune'),
 ('Samir Vishniakov', '/samir_vishniakov'),
 ('Владислав Плотников', '/donatworry'),
 ('Эдуар

Отлично! При желании ссылки на профили можно сделать полными и кликабельными:

In [29]:
people_new = []
for p in people:
    new_link =  "https://vk.com" + p[1] 
    people_new.append((p[0], new_link)) 

Проверяем:

In [30]:
people_new[0]

('Артемий Лебедев', 'https://vk.com/temalebedev')

Осталась одна проблема – мы извлекли далеко не все результаты, а только те результаты, которые «видит» браузер, то есть то, что мы как пользователи видим до скроллинга. Библиотека `selenium` умеет скроллить страницы, точнее, активировать запуск кода на JavaScript, который отвечает за скроллинг. В общем виде строка с кодом для скроллинга выглядит так (`Y` – на сколько пикселей нужно проскроллить):

    br.execute_script("window.scrollTo(0, Y)") 
    
Если нужно проскроллить до конца страницы, то тогда вместо `Y` нужно вписать значение, которое извлекается из тела документа HTML:

     document.body.scrollHeight

Чтобы извлечь все результаты и проскроллить всё до самого конца, нам понадобится цикл. 
Аглоритм такой: 

1. Выгружаем исходный код страницы.
2. Находим в этом коде необходимые элементы – извлекаем имена и ссылки на профили.
3. Скроллим вниз до конца страницы.
4. Повторяем пункты 1-3 до тех пор пока не достигнем последнего результата.

Тут может возникнуть логичный вопрос: почему нельзя сразу проскроллить страницу до самого последнего результата, а потом собрать все данные? Причина проста: раз `webdriver` «видит» браузер глазами пользователя, при скроллинге вниз более ранние результаты будут пролистываться и уходить наверх, а значит, доступа к ним уже не будет. Поэтому нужно действовать поэтапно: проскроллили – сразу выгрузили информацию, ещё проскроллили – снова выгрузили.

Для упрощения задачи напишем функцию, которая сохраняет исходный код страницы, обращаясь к драйверу `br`, и извлекает информацию о пользователях.

In [31]:
def get_users_info(br):
    html = br.page_source
    soup = BeautifulSoup(html)
    divs = soup.find_all("div", {"class" : "info"})
    
    people = []
    for d in divs:
        p = get_person(d)
        people.append(p)
        
    return people

Осталось запустить долгий цикл и на каждой итерации добавлять новые результаты после скроллинга. Импортируем функцию `sleep`, чтобы выставлять задержки между скроллингами:

In [32]:
from time import sleep

Теперь напишем код (адаптированный ответ с [StackOverflow](https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python)) для цикла. Какой тип цикла нам нужен? Нам нужен цикл, который умеет повторять операции до тех пор, пока мы не дойдём до последнего результата, то есть до того момента, когда скроллить будет некуда. Воспользуемся конструкцией `while True`, бесконечным вариантом цикла `while`, который будет запускаться до тех пор, пока не дойдёт до кода с оператором `break` (выход из цикла) или не столкнётся с ошибкой. 

In [34]:
all_results = []
last_height = br.execute_script("return document.body.scrollHeight")

while True:
    res = get_users_info(br)
    all_results.extend(res) 
    
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = br.execute_script("return document.body.scrollHeight")
    sleep(1)
    
    if new_height == last_height:
        break
        
    last_height = new_height

Пояснения к коду.

1. Сохраняем в переменную `last_height` величину, на которую мы можем проскроллить страницу за один раз в данный момент времени, чтобы дойти до конца страницы. 

2. На каждой итерации цикла `while` мы выгружаем информацию, добавляем её в список `all_results` и скроллим страницу до самого низа. После скроллинга проверяем, на сколько ещё можно проскроллить, сохраняем полученное значение в `new_height`. 

3. Если скроллить уже некуда, если мы находимся в самом низу «бесконечной» страницы с результатами поиска, то `new_height` совпадает с `last_height`. Значит, нам нужно остановить исполнение кода – выходим из цикла с помощью `break`. 

4. Если мы ещё не закончили скроллить, обновляем значение `last_height`, заменяя его на `new_height` (теперь уже в нём хранится величина, на которую мы можем проскроллить страницу за один раз в данный момент времени). Продолжаем выполнять выгрузку информации и скроллинг.

После недолгой работы кода с ячейке выше (у меня исполнение заняло примерно 2.5 минуты), мы получаем список `all_results` с именами пользователей и ссылками на их профили.

In [38]:
# 26297 результатов

len(all_results)

26297

Преобразуем список в датафрейм:

In [39]:
import pandas as pd

In [42]:
dat = pd.DataFrame(all_results)
dat.head(5) # первые 5 строк

,0,1
0,Артемий Лебедев,/temalebedev
1,Jama Buldozer,/jama_buldozer
2,Павел Дуров,/durov
3,Вован Тарасов,/id579521433
4,Сергей Крылов,/id11010073


Переименуем столбцы и сделаем ссылки на профили полными:

In [43]:
dat.columns = ["name", "link"]

In [44]:
dat["link_full"] = dat["link"].apply(lambda x: "https://vk.com/" +  x)

In [46]:
dat.head(5)

,name,link,link_full
0,Артемий Лебедев,/temalebedev,https://vk.com//temalebedev
1,Jama Buldozer,/jama_buldozer,https://vk.com//jama_buldozer
2,Павел Дуров,/durov,https://vk.com//durov
3,Вован Тарасов,/id579521433,https://vk.com//id579521433
4,Сергей Крылов,/id11010073,https://vk.com//id11010073


Теперь можно писать другой код, код, который проходит по ссылкам на профили и выгружает оттуда доступную всем информацию :) Но это уже другая история, в следующий раз попробуем поработать с ВКонтакте уже с помощью API.